# Running the relation model
In this notebook we will run and evalute the relation model proposed in the research paper [A Frustratingly Easy Approach for Entity and Relation Extraction](https://arxiv.org/pdf/2010.12812.pdf).

This is a reproduction based on the instructions left by the authors in their [GitHub repo](https://github.com/princeton-nlp/PURE)

**Environment information**

- Windows 11
- Python 3.6.13
- pip 21.2.2

## Basic setup
Firstly, we setup our notebook by importing needed libraries and modules.

And we initialize a logger.

In [1]:
from transformers import BertModel, BertPreTrainedModel
from transformers import AlbertModel, AlbertPreTrainedModel

from transformers import AutoTokenizer
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers.file_utils import PYTORCH_PRETRAINED_BERT_CACHE, WEIGHTS_NAME, CONFIG_NAME

import os
import time
import torch
import logging
import json
import numpy as np
import random
from torch.utils.data import DataLoader, TensorDataset
from torch import nn
from torch.nn import CrossEntropyLoss


logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)
logger = logging.getLogger('run_relation')

C:\Users\odaim\anaconda3\envs\PUREReprodcution\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Main classes
The authors have implemented their system in an OOP style. Let's go through the defined classes and understand what they do.

### BertForRelation
This Python class is a custom implementation of a relation classification model based on the BERT (Bidirectional Encoder Representations from Transformers) architecture. Let me break down the key components:
This class inherits from BertPreTrainedModel. Therefore, it's building on top of an existing BERT model. This is done to leverage th pre-trained BERT model's weights and then fine-tune it for our specific task.

The BertForRelation class is designed for relation classification tasks, where the goal is to predict the relationship between entities in a given text. It leverages the BERT (Bidirectional Encoder Representations from Transformers) architecture and extends the BertPreTrainedModel class, building upon a pre-trained BERT model.

The constructor initializes various components of the model:

- config: BERT model configuration.
- num_rel_labels: Number of distinct relation labels.

The setup includes:

- bert: The BERT model initialized with the provided configuration.
- dropout: A dropout layer with a dropout probability specified in the BERT configuration.
- layer_norm: Layer normalization applied to the concatenated output of the subject and object representations.
- classifier: A linear layer for classification, mapping the concatenated representation to the output logits.
- init_weights(): Initialization of the model's weights.

We must note the setup of the model:

```python
self.classifier = nn.Linear(config.hidden_size * 2, self.num_labels)
```

The model is a linear layer that takes the concatenated representation of subject and object entities as input and produces logits for each possible relation label.


- It's defined using nn.Linear and consists of two layers.
- Input dimension (input_dim) is set to the sum of the following:
  - config.hidden_size: corresponds to the hidden size of the BERT model. In BERT, each token in the input sequence is associated with a hidden vector of this size.
  - * 2: The * 2 indicates that the representations of the subject and object entities are concatenated. So, the input dimension is twice the hidden size. This is likely because the model wants to capture information from both the start and end embeddings of the entities.

Output Layer:

`self.num_labels` is the number of distinct relation labels that the model is designed to classify. Each output neuron in the linear layer corresponds to a specific relation label.

The overall architecture is a simple linear transformation that maps the concatenated representation of subject and object entities to a vector of logits, where each element of the vector corresponds to the model's prediction for a specific relation label. This linear layer is typically followed by a softmax activation during training to convert the logits into probabilities and compute the cross-entropy loss.

In [2]:
class BertForRelation(BertPreTrainedModel):
    def __init__(self, config, num_rel_labels):
        super(BertForRelation, self).__init__(config)
        self.num_labels = num_rel_labels
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.layer_norm = BertLayerNorm(config.hidden_size * 2)
        self.classifier = nn.Linear(config.hidden_size * 2, self.num_labels)
        self.init_weights()

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None, sub_idx=None, obj_idx=None, input_position=None):
        outputs = self.bert(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask, output_hidden_states=False, output_attentions=False, position_ids=input_position)
        sequence_output = outputs[0]
        sub_output = torch.cat([a[i].unsqueeze(0) for a, i in zip(sequence_output, sub_idx)])
        obj_output = torch.cat([a[i].unsqueeze(0) for a, i in zip(sequence_output, obj_idx)])
        rep = torch.cat((sub_output, obj_output), dim=1)
        rep = self.layer_norm(rep)
        rep = self.dropout(rep)
        logits = self.classifier(rep)

        if labels is not None:
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            return loss
        else:
            return logits

### AlbertForRelation

There's also the class AlbertForEntity. Which is very similar to the BertForEntity with a key difference in the underlying transformer architecture they use: BertForRelation is based on BERT, while AlbertForRelation is based on ALBERT.

In [3]:
class AlbertForRelation(AlbertPreTrainedModel):
    def __init__(self, config, num_rel_labels):
        super(AlbertForRelation, self).__init__(config)
        self.num_labels = num_rel_labels
        self.albert = AlbertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.layer_norm = BertLayerNorm(config.hidden_size * 2)
        self.classifier = nn.Linear(config.hidden_size * 2, self.num_labels)
        self.init_weights()

    def forward(self, input_ids, token_type_ids=None, attention_mask=None, labels=None, sub_idx=None, obj_idx=None):
        outputs = self.albert(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask, output_hidden_states=False, output_attentions=False)
        sequence_output = outputs[0]
        sub_output = torch.cat([a[i].unsqueeze(0) for a, i in zip(sequence_output, sub_idx)])
        obj_output = torch.cat([a[i].unsqueeze(0) for a, i in zip(sequence_output, obj_idx)])
        rep = torch.cat((sub_output, obj_output), dim=1)
        rep = self.layer_norm(rep)
        rep = self.dropout(rep)
        logits = self.classifier(rep)

        if labels is not None:
            loss_fct = CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            return loss
        else:
            return logits

### Dataset

Represents a dataset. It's a convenient wrapper for handling datasets, reading data from JSON files, and creating Document objects.

Let's break down its components:
#### `__init__` method:

- Initializes an instance of the Dataset class.
- Takes three parameters: `json_file`, `pred_file` **(default is `None`)**, and `doc_range` **(default is `None`)**.
- Reads data from the specified JSON files (`json_file` and `pred_file`).
- If a document range (`doc_range`) is provided, it selects a subset of documents within that range.
- Creates a list of Document objects based on the read data.

#### `update_from_js` method:

- Updates the dataset with new data (`js`).
- Re-creates the list of `Document` objects based on the updated data.

#### \`_read method`:

- Reads data from JSON files (`json_file` and optionally `pred_file`).
If `pred_file` is provided, it merges the data from the gold (`gold_docs`) and predicted (`pred_docs`) files.
- Returns the merged list of documents.

#### `__getitem__` method:

- Enables indexing of the dataset. Given an index `ix`, it returns the corresponding Document object.

#### `__len__` method:

- Returns the length of the dataset, i.e., the number of documents.

In [4]:
class Dataset:
    def __init__(self, json_file, pred_file=None, doc_range=None):
        self.js = self._read(json_file, pred_file)
        if doc_range is not None:
            self.js = self.js[doc_range[0]:doc_range[1]]
        self.documents = [Document(js) for js in self.js]

    def update_from_js(self, js):
        self.js = js
        self.documents = [Document(js) for js in self.js]

    def _read(self, json_file, pred_file=None):
        gold_docs = [json.loads(line) for line in open(json_file)]
        if pred_file is None:
            return gold_docs

        pred_docs = [json.loads(line) for line in open(pred_file)]
        merged_docs = []
        for gold, pred in zip(gold_docs, pred_docs):
            assert gold["doc_key"] == pred["doc_key"]
            assert gold["sentences"] == pred["sentences"]
            merged = copy.deepcopy(gold)
            for k, v in pred.items():
                if "predicted" in k:
                    merged[k] = v
            merged_docs.append(merged)

        return merged_docs

    def __getitem__(self, ix):
        return self.documents[ix]

    def __len__(self):
        return len(self.documents)

### Document

This class represents a document. It encapsulates information about a document, its sentences, and any associated clusters. It provides methods for accessing and manipulating this information.

Let's go through each part:

#### `__init__` method:

- Initializes an instance of the Document class.
- Takes a JSON object (`js`) as input.
- Extracts the document key (`_doc_key`) from the JSON object.
- Uses the `fields_to_batches` function to extract specific fields from the JSON and create a list of entries.
- Computes sentence lengths and starts to facilitate sentence indexing.
- Creates a list of Sentence objects based on the entries.
- If "`clusters`" or "`predicted_clusters`" are present in the JSON, creates lists of Cluster objects for clusters and predicted clusters.

#### `__repr__` method:

- Returns a string representation of the document, including sentence indices and their corresponding text.

#### `__getitem__` method:

- Enables indexing of the document. Given an index ix, it returns the corresponding `Sentence` object.

#### `__len__` method:

- Returns the number of sentences in the document.

#### `print_plaintext method`:

- Prints the plaintext representation of the document, sentence by sentence.

#### `find_cluster` method:

- Searches through reference clusters (either predicted or actual) to find the one containing a specified entity.
- Returns the found cluster or None if no match is found.

#### `n_tokens property`:

- Returns the total number of tokens in the document by summing the number of tokens in each sentence.

In [5]:
class Document:
    def __init__(self, js):
        self._doc_key = js["doc_key"]
        entries = fields_to_batches(js, ["doc_key", "clusters", "predicted_clusters", "section_starts"])
        sentence_lengths = [len(entry["sentences"]) for entry in entries]
        sentence_starts = np.cumsum(sentence_lengths)
        sentence_starts = np.roll(sentence_starts, 1)
        sentence_starts[0] = 0
        self.sentence_starts = sentence_starts
        self.sentences = [Sentence(entry, sentence_start, sentence_ix)
                          for sentence_ix, (entry, sentence_start)
                          in enumerate(zip(entries, sentence_starts))]
        if "clusters" in js:
            self.clusters = [Cluster(entry, i, self)
                             for i, entry in enumerate(js["clusters"])]
        if "predicted_clusters" in js:
            self.predicted_clusters = [Cluster(entry, i, self)
                                       for i, entry in enumerate(js["predicted_clusters"])]

    def __repr__(self):
        return "\n".join([str(i) + ": " + " ".join(sent.text) for i, sent in enumerate(self.sentences)])

    def __getitem__(self, ix):
        return self.sentences[ix]

    def __len__(self):
        return len(self.sentences)

    def print_plaintext(self):
        for sent in self:
            print(" ".join(sent.text))


    def find_cluster(self, entity, predicted=True):
        """
        Search through erence clusters and return the one containing the query entity, if it's
        part of a cluster. If we don't find a match, return None.
        """
        clusters = self.predicted_clusters if predicted else self.clusters
        for clust in clusters:
            for entry in clust:
                if entry.span == entity.span:
                    return clust

        return None

    @property
    def n_tokens(self):
        return sum([len(sent) for sent in self.sentences])

### Cluster

The `Cluster` class represents a cluster of entities within a document. It is used to group together entities that belong to the same cluster, providing information about the members of the cluster.
This class is designed to encapsulate information about a cluster of entities within a document. It facilitates the organization and representation of entities belonging to the same cluster.

#### Initialization
- Parameters:
  - `cluster`: A list of entries representing the entities in the cluster.
  - `cluster_id`: An identifier for the cluster.
  - `document`: The document object to which the cluster belongs.
- Attributes
  - `members`: A list of ClusterMember instances, each representing an entity in the cluster.
  - `cluster_id`: The identifier for the cluster.
  
#### Initialization Details
- The `__init__` method initializes the cluster by extracting information about each entity in the cluster.
- For each entry in the cluster, it determines the corresponding sentence and span in the document.
- It creates `ClusterMember` instances for each entity and appends them to the members list.

#### Representation
`__repr__` Method:
Returns a string representation of the cluster, including the cluster identifier and a representation of its members.
Accessing Members

`__getitem__` Method:
Allows accessing individual members of the cluster using indexing.

In [6]:
class Cluster:
    def __init__(self, cluster, cluster_id, document):
        members = []
        for entry in cluster:
            sentence_ix = get_sentence_of_span(entry, document.sentence_starts, document.n_tokens)
            sentence = document[sentence_ix]
            span = Span(entry[0], entry[1], sentence.text, sentence.sentence_start)
            ners = [x for x in sentence.ner if x.span == span]
            assert len(ners) <= 1
            ner = ners[0] if len(ners) == 1 else None
            to_append = ClusterMember(span, ner, sentence, cluster_id)
            members.append(to_append)

        self.members = members
        self.cluster_id = cluster_id

    def __repr__(self):
        return f"{self.cluster_id}: " + self.members.__repr__()

    def __getitem__(self, ix):
        return self.members[ix]


### ClusterMember

Represents an individual entity within a cluster. It provides information about the entity's span, associated named entity recognition (NER) information, the sentence it belongs to, and the identifier of the cluster to which it is assigned. The class serves as a container for information about an individual entity within a cluster. It encapsulates details such as the span, NER information, sentence context, and the cluster to which the entity is assigned.

#### Initialization
- Parameters:
  - `span`: A Span instance representing the span of the entity in the document.
  - `ner`: A NER instance representing the NER information for the entity.
  - `sentence`: A Sentence instance representing the sentence containing the entity.
  - `cluster_id`: The identifier of the cluster to which the entity belongs.
- Attributes
  - `span`: A Span instance representing the span of the entity.
  - `ner`: A NER instance representing the NER information for the entity.
  - `sentence`: A Sentence instance representing the sentence containing the entity.
  - `cluster_id`: The identifier of the cluster to which the entity belongs.
  
#### Initialization Details

The `__init__` method initializes a `ClusterMember` by assigning values to its attributes based on the provided parameters.
Representation

`__repr__` Method:
Returns a string representation of the `ClusterMember`, including the sentence index and a representation of its span.


In [7]:
class ClusterMember:
    def __init__(self, span, ner, sentence, cluster_id):
        self.span = span
        self.ner = ner
        self.sentence = sentence
        self.cluster_id = cluster_id

    def __repr__(self):
        return f"<{self.sentence.sentence_ix}> " + self.span.__repr__()


### Sentence

This class represents a sentence. It encapsulates information about a sentence, including its text and associated entities (NER, relations, events). It provides methods for accessing and manipulating this information.

Let's go through each method:

#### `__init__` method:

- Initializes an instance of the `Sentence` class.
- Takes an `entry`, `sentence_start` (index of the sentence start in the document), and `sentence_ix` (sentence index) as input.
- Stores information about the sentence's start position, text, and index.
- Parses gold entities (NER, relations, events) and predicted entities (NER, relations, events).
- Stores top spans if available.

#### `__repr__` method:

- Returns a string representation of the sentence, including the text and token indices.

#### `__len__` method:

-Returns the number of tokens in the sentence.

#### `get_flavor` method:

- Given an argument (presumably an entity), retrieves its flavor from the gold NER annotations.
- If multiple NER annotations are found for the same span, prints a message (debugging).

In [8]:
class Sentence:
    def __init__(self, entry, sentence_start, sentence_ix):
        self.sentence_start = sentence_start
        self.text = entry["sentences"]
        self.sentence_ix = sentence_ix
        # Gold
        if "ner_flavor" in entry:
            self.ner = [NER(this_ner, self.text, sentence_start, flavor=this_flavor)
                        for this_ner, this_flavor in zip(entry["ner"], entry["ner_flavor"])]
        elif "ner" in entry:
            self.ner = [NER(this_ner, self.text, sentence_start)
                        for this_ner in entry["ner"]]
        if "relations" in entry:
            self.relations = [Relation(this_relation, self.text, sentence_start) for
                              this_relation in entry["relations"]]
        if "events" in entry:
            self.events = Events(entry["events"], self.text, sentence_start)

        # Predicted
        if "predicted_ner" in entry:
            self.predicted_ner = [NER(this_ner, self.text, sentence_start, flavor=None) for
                                  this_ner in entry["predicted_ner"]]
        if "predicted_relations" in entry:
            self.predicted_relations = [Relation(this_relation, self.text, sentence_start) for
                                        this_relation in entry["predicted_relations"]]
        if "predicted_events" in entry:
            self.predicted_events = Events(entry["predicted_events"], self.text, sentence_start)

        # Top spans
        if "top_spans" in entry:
            self.top_spans = [NER(this_ner, self.text, sentence_start, flavor=None) for
                                this_ner in entry["top_spans"]]

    def __repr__(self):
        the_text = " ".join(self.text)
        the_lengths = np.array([len(x) for x in self.text])
        tok_ixs = ""
        for i, offset in enumerate(the_lengths):
            true_offset = offset if i < 10 else offset - 1
            tok_ixs += str(i)
            tok_ixs += " " * true_offset

        return the_text + "\n" + tok_ixs

    def __len__(self):
        return len(self.text)

    def get_flavor(self, argument):
        the_ner = [x for x in self.ner if x.span == argument.span]
        if len(the_ner) > 1:
            print("Weird")
        if the_ner:
            the_flavor = the_ner[0].flavor
        else:
            the_flavor = None
        return the_flavor

### NER

The NER class represents a Named Entity Recognition annotation within a sentence. It encapsulates information about a named entity, providing methods for representation and equality checks. The `Span` (we'll get to it next) class is used to represent the span of the named entity within the context of the sentence.

Here are some details about it:
#### Initialization

- Parameters:
  - `ner`: A list containing information about the NER span, including start index, end index, and label.
  - `text`: The text content of the sentence.
  - `sentence_start`: The index of the sentence start in the document.
  - `flavor`: An optional parameter representing the flavor or type of the named entity.
  
- Attributes:
  - `span`: An instance of the Span class representing the span of the NER annotation.
  - `label`: The label assigned to the NER entity.
  - `flavor`: The flavor or type of the named entity.
  
- Representation

`__repr__` Method:
Returns a string representation of the NER annotation, including the span and label.
Equality Check

`__eq__` Method:
Checks if two NER instances are equal by comparing their span, label, and flavor.

In [9]:
class NER:
    def __init__(self, ner, text, sentence_start, flavor=None):
        self.span = Span(ner[0], ner[1], text, sentence_start)
        self.label = ner[2]
        self.flavor = flavor

    def __repr__(self):
        return self.span.__repr__() + ": " + self.label

    def __eq__(self, other):
        return (self.span == other.span and
                self.label == other.label and
                self.flavor == other.flavor)

### Span

The `Span` class represents a span of text within a document. It encapsulate information about a text span, providing methods for representation, equality checks, and hashing. It ensures the proper handling of spans within the document and sentence contexts.

#### Initialization
- Parameters:
  - `start`: The starting index of the span in the entire document.
  - `end`: The ending index of the span in the entire document.
  - `text`: The text content of the entire document.
  - `sentence_start`: The index of the sentence start in the document.
  
- Attributes
  - `start_doc`, `end_doc`: The start and end indices of the span in the entire document.
  - `span_doc`: A tuple representing the span in the entire document.
  - `start_sent`, `end_sent`: The start and end indices of the span within the sentence.
  - `span_sent`: A tuple representing the span within the sentence.
  - `text`: The actual text content of the span.
  
#### Representation

`__repr__` Method:
Returns a string representation of the span, including start and end indices and the actual text content.
Equality Check

`__eq__` Method:
Checks if two Span instances are equal by comparing their spans in both the document and the sentence, along with the text content.
Hashing

`__hash__` Method:
Computes a hash value for the Span instance based on its document and sentence spans, as well as the text content.

In [10]:
class Span:
    def __init__(self, start, end, text, sentence_start):
        self.start_doc = start
        self.end_doc = end
        self.span_doc = (self.start_doc, self.end_doc)
        self.start_sent = start - sentence_start
        self.end_sent = end - sentence_start
        self.span_sent = (self.start_sent, self.end_sent)
        self.text = text[self.start_sent:self.end_sent + 1]

    def __repr__(self):
        return str((self.start_sent, self.end_sent, self.text))

    def __eq__(self, other):
        return (self.span_doc == other.span_doc and
                self.span_sent == other.span_sent and
                self.text == other.text)

    def __hash__(self):
        tup = self.span_doc + self.span_sent + (" ".join(self.text),)
        return hash(tup)


### Relation

The Relation class is designed to represent a relation between two spans within a text. It encapsulates information about the spans, such as their start and end positions, the text they cover, and the label assigned to the relation. Here's a breakdown of the class:

#### Initialization

- Parameters:

  - `relation`: A tuple representing the start and end positions of the two spans and the label of the relation.
  - `text`: The text containing the spans.
  - `sentence_start`: The starting position of the sentence in the text.
  
- Attributes

  - `pair`: A tuple containing two Span objects (span1 and span2) representing the spans of the relation.
  - `label`: The label of the relation.
  
#### Representation
`__repr__` Method: Returns a string representation of the Relation object, including the string representations of the two spans, the relation between them (", "), and the label.

`__eq__` Method: Checks if two Relation objects are equal by comparing their span pairs and labels.

In [11]:
class Relation:
    def __init__(self, relation, text, sentence_start):
        start1, end1 = relation[0], relation[1]
        start2, end2 = relation[2], relation[3]
        label = relation[4]
        span1 = Span(start1, end1, text, sentence_start)
        span2 = Span(start2, end2, text, sentence_start)
        self.pair = (span1, span2)
        self.label = label

    def __repr__(self):
        return self.pair[0].__repr__() + ", " + self.pair[1].__repr__() + ": " + self.label

    def __eq__(self, other):
        return (self.pair == other.pair) and (self.label == other.label)

### InputFeatures

This class represents a single set of features for the relation model. It encapsulates the necessary information required for processing a single example during training or inference. 

In [12]:
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id, sub_idx, obj_idx):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id
        self.sub_idx = sub_idx
        self.obj_idx = obj_idx


## Utility functions
The authors have aslo implemented some utility functions. Let's go through them and understand what they do.

### add_marker_tokens
This function, as discussed by the authors, extends the tokenizer's vocabulary with special marker tokens.

Therfore, providing additional information about the structure and types of entities in the input sequences.

In [13]:
def add_marker_tokens(tokenizer, ner_labels):
    new_tokens = ['<SUBJ_START>', '<SUBJ_END>', '<OBJ_START>', '<OBJ_END>']
    for label in ner_labels:
        new_tokens.append('<SUBJ_START=%s>'%label)
        new_tokens.append('<SUBJ_END=%s>'%label)
        new_tokens.append('<OBJ_START=%s>'%label)
        new_tokens.append('<OBJ_END=%s>'%label)
    for label in ner_labels:
        new_tokens.append('<SUBJ=%s>'%label)
        new_tokens.append('<OBJ=%s>'%label)
    tokenizer.add_tokens(new_tokens)
    logger.info('# vocab after adding markers: %d'%len(tokenizer))

### convert_examples_to_features

The convert_examples_to_features function is responsible for converting the list of input examples into a list of InputFeatures objects, which can be used as input to the relation classification. It tokenizes the input examples, adds special markers for subject and object entities, handles token length constraints, and prepares the data for model input.

**Let's break down the key components of this function:**

#### Input Parameters:
- `examples`: A list of input examples, where each example is a dictionary containing information about tokens, subject and object entities, and the relation.
- `label2id`: A dictionary mapping relation labels to their corresponding integer IDs.
- `max_seq_length`: The maximum sequence length allowed for the input tokens.
- `tokenizer`: The tokenizer used to tokenize input sequences.
- `special_tokens`: A dictionary used to keep track of special tokens introduced during processing.
- `unused_tokens`: A boolean indicating whether tokens should be used as special tokens.

In [14]:
def convert_examples_to_features(examples, label2id, max_seq_length, tokenizer, special_tokens, unused_tokens=True):
    """
    Loads a data file into a list of `InputBatch`s.
    unused_tokens: whether use [unused1] [unused2] as special tokens
    """

    def get_special_token(w):
        if w not in special_tokens:
            if unused_tokens:
                special_tokens[w] = "[unused%d]" % (len(special_tokens) + 1)
            else:
                special_tokens[w] = ('<' + w + '>').lower()
        return special_tokens[w]

    num_tokens = 0
    max_tokens = 0
    num_fit_examples = 0
    num_shown_examples = 0
    features = []
    for (ex_index, example) in enumerate(examples):
        if ex_index % 10000 == 0:
            logger.info("Writing example %d of %d" % (ex_index, len(examples)))

        tokens = [CLS]
        SUBJECT_START = get_special_token("SUBJ_START")
        SUBJECT_END = get_special_token("SUBJ_END")
        OBJECT_START = get_special_token("OBJ_START")
        OBJECT_END = get_special_token("OBJ_END")
        SUBJECT_NER = get_special_token("SUBJ=%s" % example['subj_type'])
        OBJECT_NER = get_special_token("OBJ=%s" % example['obj_type'])

        SUBJECT_START_NER = get_special_token("SUBJ_START=%s"%example['subj_type'])
        SUBJECT_END_NER = get_special_token("SUBJ_END=%s"%example['subj_type'])
        OBJECT_START_NER = get_special_token("OBJ_START=%s"%example['obj_type'])
        OBJECT_END_NER = get_special_token("OBJ_END=%s"%example['obj_type'])

        for i, token in enumerate(example['token']):
            if i == example['subj_start']:
                sub_idx = len(tokens)
                tokens.append(SUBJECT_START_NER)
            if i == example['obj_start']:
                obj_idx = len(tokens)
                tokens.append(OBJECT_START_NER)
            for sub_token in tokenizer.tokenize(token):
                tokens.append(sub_token)
            if i == example['subj_end']:
                tokens.append(SUBJECT_END_NER)
            if i == example['obj_end']:
                tokens.append(OBJECT_END_NER)
        tokens.append(SEP)

        num_tokens += len(tokens)
        max_tokens = max(max_tokens, len(tokens))

        if len(tokens) > max_seq_length:
            tokens = tokens[:max_seq_length]
            if sub_idx >= max_seq_length:
                sub_idx = 0
            if obj_idx >= max_seq_length:
                obj_idx = 0
        else:
            num_fit_examples += 1

        segment_ids = [0] * len(tokens)
        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        input_mask = [1] * len(input_ids)
        padding = [0] * (max_seq_length - len(input_ids))
        input_ids += padding
        input_mask += padding
        segment_ids += padding
        label_id = label2id[example['relation']]
        assert len(input_ids) == max_seq_length
        assert len(input_mask) == max_seq_length
        assert len(segment_ids) == max_seq_length

        if num_shown_examples < 20:
            if (ex_index < 5) or (label_id > 0):
                num_shown_examples += 1
                logger.info("*** Example ***")
                logger.info("guid: %s" % (example['id']))
                logger.info("tokens: %s" % " ".join(
                        [str(x) for x in tokens]))
                logger.info("input_ids: %s" % " ".join([str(x) for x in input_ids]))
                logger.info("input_mask: %s" % " ".join([str(x) for x in input_mask]))
                logger.info("segment_ids: %s" % " ".join([str(x) for x in segment_ids]))
                logger.info("label: %s (id = %d)" % (example['relation'], label_id))
                logger.info("sub_idx, obj_idx: %d, %d" % (sub_idx, obj_idx))

        features.append(
                InputFeatures(input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids,
                              label_id=label_id,
                              sub_idx=sub_idx,
                              obj_idx=obj_idx))
    logger.info("Average #tokens: %.2f" % (num_tokens * 1.0 / len(examples)))
    logger.info("Max #tokens: %d"%max_tokens)
    logger.info("%d (%.2f %%) examples can fit max_seq_length = %d" % (num_fit_examples,
                num_fit_examples * 100.0 / len(examples), max_seq_length))
    return features


### setseed
The setseed function sets the random seed for various random number generators in the environment, specifically for the `random` module, `numpy` library, and the `PyTorch` library.

Setting a random seed ensures reproducibility in the generation of random numbers, which is crucial for obtaining consistent results when running experiments or training machine learning models.

In [15]:
def setseed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

### simple_accuracy
This function calculates the accuracy of a model's predictions by comparing the predicted labels (`preds`) with the actual ground truth labels (`labels`)

In [16]:
def simple_accuracy(preds, labels):
    return (preds == labels).mean()

### compute_f1

This function is used to comute the F1 score of the model.

In [17]:
def compute_f1(preds, labels, e2e_ngold):
    n_gold = n_pred = n_correct = 0
    for pred, label in zip(preds, labels):
        if pred != 0:
            n_pred += 1
        if label != 0:
            n_gold += 1
        if (pred != 0) and (label != 0) and (pred == label):
            n_correct += 1
    if n_correct == 0:
        return {'precision': 0.0, 'recall': 0.0, 'f1': 0.0}
    else:
        prec = n_correct * 1.0 / n_pred
        recall = n_correct * 1.0 / n_gold
        if prec + recall > 0:
            f1 = 2.0 * prec * recall / (prec + recall)
        else:
            f1 = 0.0

        if e2e_ngold is not None:
            e2e_recall = n_correct * 1.0 / e2e_ngold
            e2e_f1 = 2.0 * prec * e2e_recall / (prec + e2e_recall)
        else:
            e2e_recall = e2e_f1 = 0.0
        return {'precision': prec, 'recall': e2e_recall, 'f1': e2e_f1, 'task_recall': recall, 'task_f1': f1, 
        'n_correct': n_correct, 'n_pred': n_pred, 'n_gold': e2e_ngold, 'task_ngold': n_gold}


### evaluate

Evaluates a trained model on an evaluation dataset. It computes various evaluation metrics, including loss, F1 score, accuracy, and additional metrics provided by the compute_f1 function.

In [18]:
def evaluate(model, device, eval_dataloader, eval_label_ids, num_labels, e2e_ngold=None, verbose=True):
    model.eval()
    eval_loss = 0
    nb_eval_steps = 0
    preds = []
    for input_ids, input_mask, segment_ids, label_ids, sub_idx, obj_idx in eval_dataloader:
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)
        label_ids = label_ids.to(device)
        sub_idx = sub_idx.to(device)
        obj_idx = obj_idx.to(device)
        with torch.no_grad():
            logits = model(input_ids, segment_ids, input_mask, labels=None, sub_idx=sub_idx, obj_idx=obj_idx)
        loss_fct = CrossEntropyLoss()
        tmp_eval_loss = loss_fct(logits.view(-1, num_labels), label_ids.view(-1))
        eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1
        if len(preds) == 0:
            preds.append(logits.detach().cpu().numpy())
        else:
            preds[0] = np.append(
                preds[0], logits.detach().cpu().numpy(), axis=0)

    eval_loss = eval_loss / nb_eval_steps
    logits = preds[0]
    preds = np.argmax(preds[0], axis=1)
    result = compute_f1(preds, eval_label_ids.numpy(), e2e_ngold=e2e_ngold)
    result['accuracy'] = simple_accuracy(preds, eval_label_ids.numpy())
    result['eval_loss'] = eval_loss
    if verbose:
        logger.info("***** Eval results *****")
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
    return preds, result, logits


### print_pred_json

This function is used for formatting and printing the predicted relations based on the model's predictions.

In [19]:
def print_pred_json(eval_data, eval_examples, preds, id2label, output_file):
    rels = dict()
    for ex, pred in zip(eval_examples, preds):
        doc_sent, sub, obj = decode_sample_id(ex['id'])
        if doc_sent not in rels:
            rels[doc_sent] = []
        if pred != 0:
            rels[doc_sent].append([sub[0], sub[1], obj[0], obj[1], id2label[pred]])

    js = eval_data.js
    for doc in js:
        doc['predicted_relations'] = []
        for sid in range(len(doc['sentences'])):
            k = '%s@%d'%(doc['doc_key'], sid)
            doc['predicted_relations'].append(rels.get(k, []))
    
    logger.info('Output predictions to %s..'%(output_file))
    with open(output_file, 'w') as f:
        f.write('\n'.join(json.dumps(doc) for doc in js))


### save_trained_model

The save_trained_model function is responsible for saving the trained model, its configuration, and the associated tokenizer to the specified output directory.

In [20]:
def save_trained_model(output_dir, model, tokenizer):
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    logger.info('Saving model to %s'%output_dir)
    model_to_save = model.module if hasattr(model, 'module') else model
    output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
    output_config_file = os.path.join(output_dir, CONFIG_NAME)
    torch.save(model_to_save.state_dict(), output_model_file)
    model_to_save.config.to_json_file(output_config_file)
    tokenizer.save_vocabulary(output_dir)

### generate_relation_data
The generate_relation_data function is designed to prepare data for the training or evaluation of the relation model. It takes an entity dataset (`entity_data`), generates samples for relation extraction, and returns the dataset, a list of samples, and the total number of relations in the dataset. Let's break down the key components of this function:

#### Input Parameters:
- `entity_data`: The entity dataset from which relation data will be generated.
- `use_gold`: A boolean parameter indicating whether to use gold (true) or predicted (false) named entities and relations. Default is False.
- `context_window`: An integer representing the size of the context window to consider around each sentence. Default is 0.

This function processes the entity dataset, extracting named entities and relations. It then generates samples for relation extraction, considering optional context windows. The generated samples include information about document ID, sentence ID, relation labels, entity spans, and tokenized text. The function returns the processed dataset, the list of generated samples, and the total number of relations in the dataset. The generated samples can be used for training or evaluating a relation extraction model.

In [21]:
def generate_relation_data(entity_data, use_gold=False, context_window=0):
    """
    Prepare data for the relation model
    If training: set use_gold = True
    """
    logger.info('Generate relation data from %s'%(entity_data))
    data = Dataset(entity_data)

    nner, nrel = 0, 0
    max_sentsample = 0
    samples = []
    for doc in data:
        for i, sent in enumerate(doc):
            sent_samples = []

            nner += len(sent.ner)
            nrel += len(sent.relations)
            if use_gold:
                sent_ner = sent.ner
            else:
                sent_ner = sent.predicted_ner
            
            gold_ner = {}
            for ner in sent.ner:
                gold_ner[ner.span] = ner.label
            
            gold_rel = {}
            for rel in sent.relations:
                gold_rel[rel.pair] = rel.label
            
            sent_start = 0
            sent_end = len(sent.text)
            tokens = sent.text

            if context_window > 0:
                add_left = (context_window-len(sent.text)) // 2
                add_right = (context_window-len(sent.text)) - add_left

                j = i - 1
                while j >= 0 and add_left > 0:
                    context_to_add = doc[j].text[-add_left:]
                    tokens = context_to_add + tokens
                    add_left -= len(context_to_add)
                    sent_start += len(context_to_add)
                    sent_end += len(context_to_add)
                    j -= 1

                j = i + 1
                while j < len(doc) and add_right > 0:
                    context_to_add = doc[j].text[:add_right]
                    tokens = tokens + context_to_add
                    add_right -= len(context_to_add)
                    j += 1
            
            for x in range(len(sent_ner)):
                for y in range(len(sent_ner)):
                    if x == y:
                        continue
                    sub = sent_ner[x]
                    obj = sent_ner[y]
                    label = gold_rel.get((sub.span, obj.span), 'no_relation')
                    sample = {}
                    sample['docid'] = doc._doc_key
                    sample['id'] = '%s@%d::(%d,%d)-(%d,%d)'%(doc._doc_key, sent.sentence_ix, sub.span.start_doc, sub.span.end_doc, obj.span.start_doc, obj.span.end_doc)
                    sample['relation'] = label
                    sample['subj_start'] = sub.span.start_sent + sent_start
                    sample['subj_end'] = sub.span.end_sent + sent_start
                    sample['subj_type'] = sub.label
                    sample['obj_start'] = obj.span.start_sent + sent_start
                    sample['obj_end'] = obj.span.end_sent + sent_start
                    sample['obj_type'] = obj.label
                    sample['token'] = tokens
                    sample['sent_start'] = sent_start
                    sample['sent_end'] = sent_end

                    sent_samples.append(sample)

            max_sentsample = max(max_sentsample, len(sent_samples))
            samples += sent_samples
    
    tot = len(samples)
    logger.info('#samples: %d, max #sent.samples: %d'%(tot, max_sentsample))

    return data, samples, nrel

### decode_sample_id

Used to decode a sample identified by its ID into: document and sentence information (`doc_sent`), subject coordinates (`sub`), and object coordinates (`obj`).

In [22]:
def decode_sample_id(sample_id):
    doc_sent = sample_id.split('::')[0]
    pair = sample_id.split('::')[1]
    pair = pair.split('-')
    sub = (int(pair[0][1:-1].split(',')[0]), int(pair[0][1:-1].split(',')[1]))
    obj = (int(pair[1][1:-1].split(',')[0]), int(pair[1][1:-1].split(',')[1]))

    return doc_sent, sub, obj


### get_sentence_of_span

determine the index of the sentence to which a given span (represented as a pair of indices) belongs, and returns that index.

In [23]:
def get_sentence_of_span(span, sentence_starts, doc_tokens):
    """
    Return the index of the sentence that the span is part of.
    """
    # Inclusive sentence ends
    sentence_ends = [x - 1 for x in sentence_starts[1:]] + [doc_tokens - 1]
    in_between = [span[0] >= start and span[1] <= end
                  for start, end in zip(sentence_starts, sentence_ends)]
    assert sum(in_between) == 1
    the_sentence = in_between.index(True)
    return the_sentence

In [ ]:
def fields_to_batches(d, keys_to_ignore=[]):
    keys = [key for key in d.keys() if key not in keys_to_ignore]
    lengths = [len(d[k]) for k in keys]
    assert len(set(lengths)) == 1
    length = lengths[0]
    res = [{k: d[k][i] for k in keys} for i in range(length)]
    return res

## Running and evaluationg the pre-trained relation model

Now that the setup is out of the way. We can actually run the model and evaluate it with a pre-trained BERT-based model on the SciERC dataset.

We will do that in the run relation notebook